In [2]:
import pandas as pd

In [4]:
anime_df = pd.read_csv("cleaned_anime.csv")
anime_df.head()

,anime_id,name,genre
0,28891,Haikyuu!! Second Season,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun..."
1,23273,Shigatsu wa Kimi no Uso,"['Drama', 'Music', 'Romance', 'School', 'Shoun..."
2,34599,Made in Abyss,"['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F..."
3,5114,Fullmetal Alchemist: Brotherhood,"['Action', 'Military', 'Adventure', 'Comedy', ..."
4,31758,Kizumonogatari III: Reiketsu-hen,"['Action', 'Mystery', 'Supernatural', 'Vampire']"


In [7]:
anime_df.drop("genre", axis=1, inplace=True)
anime_df.head()

,anime_id,name
0,28891,Haikyuu!! Second Season
1,23273,Shigatsu wa Kimi no Uso
2,34599,Made in Abyss
3,5114,Fullmetal Alchemist: Brotherhood
4,31758,Kizumonogatari III: Reiketsu-hen


In [9]:
rating_df = pd.read_csv("cleaned_rating.csv")
rating_df.head()

,user,anime_id,rating
0,-----noname-----,18441,2
1,-----noname-----,2025,4
2,---SnowFlake---,1535,6
3,---was-----,10110,8
4,--EYEPATCH--,35839,10


In [11]:
from surprise import Reader
from surprise import Dataset

reader = Reader(rating_scale=(0, 10))

data = Dataset.load_from_df(rating_df[["user", "anime_id", "rating"]], reader)
data

In [12]:
from surprise import SVD

In [13]:
from surprise.model_selection import GridSearchCV

In [14]:
%%time

# Create parameters combinations
params = {
    "n_epochs": [10, 15], "lr_all": [0.003, 0.005, 0.007], "reg_all": [0.01, 0.02, 0.03]
}

# Run the grid search using SVD and the parameters to find the best parameters for Root Mean Square Error and Mean Absolute Error
gs = GridSearchCV(SVD, params, measures=['rmse', 'mae'], cv=3, joblib_verbose=2, n_jobs=-1)

gs.fit(data)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   32.1s


Wall time: 1min 4s


[Parallel(n_jobs=-1)]: Done  54 out of  54 | elapsed:  1.1min finished


In [15]:
# Best RMSE score
print(gs.best_score["rmse"])

# Best combination of parameters for the best RMSE
print(gs.best_params["rmse"])

1.9865442851376922
{'n_epochs': 15, 'lr_all': 0.007, 'reg_all': 0.03}


In [16]:
algo = gs.best_estimator["rmse"]
algo.fit(data.build_full_trainset())

In [20]:
pred = algo.predict("Vaenny", 6489).est
pred

6.724706336131116

In [21]:
from surprise.dump import dump
dump("recommender.pkl", algo)